In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import dill as pickle
import json
from IPython.display import Image
from IPython.core.display import HTML
from hazm import *
from fastai.text import *

from glob import glob
import re
from pathlib import Path


In [2]:
PATH = 'extract/AA/'

## Clean Data

This data sets is based on persian-wikipedia corpus. You can download [fawiki-20180401-pages-articles.xml.bz2 here (https://dumps.wikimedia.org/fawiki/20180401/)

In [ ]:
sample_path = f'{PATH}sample/'
sample_files = !ls {sample_path}
os.makedirs(f'{sample_path}tmp',exist_ok=True)

In [ ]:
trn_path = f'{PATH}train/'
trn_files = !ls {trn_path}
os.makedirs(f'{trn_path}tmp',exist_ok=True)

In [ ]:
val_path = f'{PATH}valid/'
val_files = !ls {val_path}
os.makedirs(f'{val_path}tmp',exist_ok=True)

In [ ]:
def clean_files(extracted_filelist,dir_path):    
    cleaned_all = []
    for ext_file in extracted_filelist:
        input_file = f'{dir_path}{ext_file}'
        with open(input_file,'r', encoding='utf-8') as f:
            raw_txt = f.readlines()
            cleaned_doc = []
            for line in raw_txt:
                new_line = re.sub('<[^<]+?>', '', line)
                new_line = re.sub('[a-zA-Z]+','',new_line)
                new_line = re.sub('\d+','',new_line)
                new_line = re.sub('[</>=":/?(\).\-,&]+','',new_line)            
                new_line = new_line.strip()
                if new_line != '':
                    cleaned_doc.append(new_line)

            new_doc = "\n".join(cleaned_doc)
            cleaned_all.append(new_doc)
            with open(f"{dir_path}tmp/{ext_file}_clean.txt", "w", encoding='utf-8') as text_file:
                text_file.write(new_doc)
    return cleaned_all


Now clean wiki files are stored in tmp subdirs in train and valid dirs.

## Prepare dataset

In [ ]:
PATH=Path('extract/AA')

In [ ]:
def get_texts(path):
    texts = []
    for fname in (path).glob('*.*'):
        texts.append(fname.open('r').read())
    return np.array(texts)

trn_texts= get_texts(PATH/'train/tmp')
val_texts = get_texts(PATH/'valid/tmp')

In [ ]:
len(trn_texts),len(val_texts)

In [ ]:
trn_str = '\n'.join(trn_texts)
val_str = '\n'.join(val_texts)

n = 1500
trn_texts = np.array([trn_str[i:i+n] for i in range(0,len(trn_str),n)])
val_texts = np.array([val_str[i:i+n] for i in range(0,len(val_str),n)])


In [ ]:
trn_texts.shape,val_texts.shape

In [ ]:
len(trn_texts),len(val_texts)

## Standardized format 

In [ ]:
LM_PATH=Path('extract/AA/persian_lm/')
LM_PATH.mkdir(exist_ok=True)

In [ ]:
col_names = ['labels','text']

In [ ]:
df_trn = pd.DataFrame({'text':trn_texts, 'labels':[0]*len(trn_texts)}, columns=col_names)
df_val = pd.DataFrame({'text':val_texts, 'labels':[0]*len(val_texts)}, columns=col_names)

df_trn.to_csv(LM_PATH/'train.csv', header=False, index=False)
df_val.to_csv(LM_PATH/'test.csv', header=False, index=False)

## Language Model tokens

In [ ]:
chunksize=24000

In [ ]:
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

In [ ]:
def get_texts(df, n_lbls=1):
    labels = df.iloc[:,range(n_lbls)].values.astype(np.int64)
    texts = df[n_lbls].astype(str)
    for i in range(n_lbls+1, len(df.columns)): texts += df[i].astype(str)
    tok = [word_tokenize(s) for s in texts]
    return tok, list(labels)

In [ ]:
def get_all(df, n_lbls):
    tok, labels = [], []
    for i, r in enumerate(df):
        print(i)
        tok_, labels_ = get_texts(r, n_lbls)
        tok += tok_;
        labels += labels_
    return tok, labels

In [ ]:
df_trn = pd.read_csv(LM_PATH/'train.csv', header=None,chunksize=chunksize)
df_val = pd.read_csv(LM_PATH/'test.csv', header=None,chunksize=chunksize)

In [ ]:
tok_trn, trn_labels = get_all(df_trn, 1)

In [ ]:
tok_val,val_labels = get_all(df_val, 1)

In [ ]:
print(' '.join(tok_trn[10000]))

In [ ]:
(LM_PATH/'tmp').mkdir(exist_ok=True)

In [ ]:
np.save(LM_PATH/'tmp'/'tok_trn_hazm.npy', tok_trn)
np.save(LM_PATH/'tmp'/'tok_val_hazm.npy', tok_val)

In [ ]:
tok_trn = np.load(LM_PATH/'tmp'/'tok_trn_hazm.npy')
tok_val = np.load(LM_PATH/'tmp'/'tok_val_hazm.npy')

In [ ]:
freq = Counter(p for o in tok_trn for p in o)
freq.most_common(25)

In [ ]:
max_vocab = 60000
min_freq = 2

In [ ]:
#int to string: list of unique tokens in the vocab
itos = [o for o,c in freq.most_common(max_vocab) if c > min_freq]
# add unique tokens for unknown and padding
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

In [ ]:
stoi = collections.defaultdict(lambda: 0, {v:k for k,v in enumerate(itos)})
len(itos)

In [ ]:
stoi['سلام']

In [ ]:
itos[5282]

In [ ]:
stoi['__unk__']

create index for every word. needed stoi dictionary to create this and also extract the word asociated with indexes.

In [ ]:
trn_lm = np.array([[stoi[o] for o in p] for p in tok_trn])
val_lm = np.array([[stoi[o] for o in p] for p in tok_val])

In [ ]:
np.save(LM_PATH/'tmp'/'trn_ids_hazm.npy', trn_lm)
np.save(LM_PATH/'tmp'/'val_ids_hazm.npy', val_lm)
pickle.dump(itos, open(LM_PATH/'tmp'/'itos_hazm.pkl', 'wb'))

In [ ]:
vs=len(itos)
vs,len(trn_lm)

## Language Model

In [3]:
LM_PATH=Path('extract/AA/persian_lm/')
LM_PATH.mkdir(exist_ok=True)

In [4]:
trn_lm = np.load(LM_PATH/'tmp'/'trn_ids_hazm.npy')
val_lm = np.load(LM_PATH/'tmp'/'val_ids_hazm.npy')
itos = pickle.load(open(LM_PATH/'tmp'/'itos_hazm.pkl', 'rb'))

In [5]:
vs=len(itos)
vs,len(trn_lm)

(60002, 188257)

In [6]:
stoi = collections.defaultdict(lambda: 0, {v:k for k,v in enumerate(itos)})
len(itos)

60002

In [7]:
stoi['سلام']

5046

In [8]:
em_sz,nh,nl = 400,1150,3

In [9]:
wd=1e-7
bptt=70
bs=52
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

In [10]:
trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
md = LanguageModelData(PATH, 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)

In [11]:
trn_lm[0][:10]

[508, 2467, 508, 2467, 12, 742, 1361, 8, 7, 2]

In [12]:
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

In [13]:
learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

In [43]:
learner.metrics = [accuracy]
learner.unfreeze()

In [44]:
lr=1e-3
lrs = lr

In [ ]:
learner.fit(lrs/2, 1, wds=wd, use_clr=(32,2), cycle_len=1)

After 45 epochs, we get validation loss of 4.01 and accuracy 0.34112.

In [ ]:
learner.load('lm_hazm_ft_after_41epochs')

In [28]:
hist8 =  learner.fit(lrs/2, 1, wds=1e-07, use_clr=(32,2),cycle_len=4)
learner.save('lm_hazm_ft_after_45epochs')

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                       
    0      4.070532   4.043118   0.337609  
    1      4.089289   4.075927   0.333111                       
    2      4.058505   4.041278   0.337166                       
    3      4.03361    4.011021   0.34112                        



In [ ]:
learner.load('lm_hazm_ft_after_45epochs')

In [30]:
hist8

[array([4.01102]), 0.3411197185286204]

In [ ]:
hist9 =  learner.fit(lrs/2, 1, wds=1e-07, use_clr=(32,2),cycle_len=8)
learner.save('lm_hazm_ft_after_53epochs')

HBox(children=(IntProgress(value=0, description='Epoch', max=8), HTML(value='')))

 99%|█████████▉| 14967/15118 [56:04<00:33,  4.45it/s, loss=3.94]

In [33]:
hist9

[array([4.00174]), 0.3424943725592021]

In [45]:
learner.load('lm_hazm_ft_after_53epochs')

In [ ]:
hist10 =  learner.fit(lrs/2, 1, wds=1e-07, use_clr=(32,2),cycle_len=8)
learner.save('lm_hazm_ft_after_61epochs')
hist10

HBox(children=(IntProgress(value=0, description='Epoch', max=8), HTML(value='')))

 20%|█▉        | 2961/15118 [10:52<44:39,  4.54it/s, loss=4.09]

In [15]:
learner.load('lm_hazm_ft_after_61epochs')

In [47]:
hist10

[array([4.00118]), 0.34272543855214127]

running more epochs does not change the accuracy.

## Test

Let's test this model to predict next work for texts of various length in Farsi.

In [16]:
m = learner.model

In [17]:
def proc_str(s): return Tokenizer().spacy_tok(s)

In [18]:
def num_str(s): 
    idx_arr = np.array([stoi[tok] for tok in proc_str(s)])
    return torch.from_numpy(np.expand_dims(idx_arr,axis=1)).cuda()

In [21]:
def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(Variable(t))
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = itos[n.data[0]]
        print(word, end=' ')
        #if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=bs

In [22]:
ss = """

رضا شاه به عنوان رهبری مستبد و مقتدر اما د رعین حال کارآمد و مفید معروف است؛ فردی که با تکیه بر منابعی محدود و وسائلی ابتدایی، راه آهن سراسری ایران را از میان کوههای صعب العبور گذر داد و در عین حال، سید حسن مدرس و شماری از نزدیکان خود چون تیمورتاش را هم به قتل رساند.

به نظرم توجه به رضا شاه نوعی نومیدی از شرایطی است که نهاد دولت را از اقتدار تهی کرده است به طوری که توان تصمیم گیری برای حل معضلات کشور و یا اجرای تصمیمات خود را ندارد و کارش به "حرف درمانی" تقلیل یافته است. در واقع با کمی تأمل می توان دریافت که کشور نه فقط دستخوش نوعی از ملوک الطوایفی است و مسئولان هر استانی ساز خود را می نوازند، بلکه در هر شهر و آبادی نیز دهها نهاد و دستگاه با تعریف منافع و رانت های مشخص اقتصادی و سیاسی و مدیریتی برای خود و اطرافیان شان، در جهت خنثی ساز فعالیت های یکدیگر در نزاع و رقابت اند و از این جهت نه فقط مشکلی را حل نمی کنند بلکه به انباشت روزافزون مشکلات دامن می زنند.


"""

In [23]:
sample_model(m,ss,l=50)

...و به ویژه در میان مردم و مردم به ویژه در میان مردم و مردم به شدت مورد انتقاد قرار می گیرد و در نتیجه به دلیل عدم وجود یک نظام حکومتی و عدم وجود یک نظام حکومتی ضعیف و ضعیف ، قدرت و نفوذ دولت در این کشور به 

In [26]:
ss="""
پیش از این هم نوشته بودم که در شرایط انباشت مشکلات  یک فرد عادی جامعه در پی دمکراسی و این قبیل سخنان نیست؛"
""" 

In [27]:
sample_model(m,ss,l=50)

...، در سال ، در یک مصاحبه با روزنامه گاردین ، در مورد اینکه آیا این موضوع را به عنوان یک فرد در جامعه مطرح می‌کند یا نه ، گفت « من در این مورد ، به عنوان یک فرد ، یک فرد را به عنوان یک فرد ، یک 

In [28]:
ss= "وای این فخرآور خیلی بانمکه که برای خودش رفته پول داده امضای دونالد ترامپ خریده و بعد به خودش از قول ترامپ گفته: توماس جفرسون ایران. می‌خوام برم یکی عینش رو سفارش بدم بگم دونالد برام بنویسه تو شهناز تهرانی آمریکایی."

In [29]:
sample_model(m,ss,l=50)

...، یک شرکت هواپیمایی با کد یاتا است که در تاریخ میلادی تأسیس شده_است دفتر مرکزی ایر در فرودگاه بین‌المللی بن لادن ، عربستان سعودی واقع شده‌است و قطب این شرکت هواپیمایی در فرودگاه بین‌المللی ملک عبدالعزیز قرار دارد و به مقصد ، پرواز مستقیم انجام می‌دهد ایر ایر ایر 